In [1]:
import numpy as np 
import cupy as cp
import djlib as dj
import pickle

In [2]:
datafile = '/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_working/eci_monte_carlo/stan_demo/sensible_parameters/50k/full_enumeration.json'
data = dj.vasputils.casm_query_reader(datafile)

eci_posterior_file = '/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_working/eci_monte_carlo/stan_demo/sensible_parameters/50k/stan_results_hierarchical.pkl'
with open(eci_posterior_file,'rb') as f:
    eci_posterior = pickle.load(f)
    eci_posterior = eci_posterior['eci']


In [3]:
data.keys()
corr = np.squeeze(np.array(data['corr']))
comp = np.reshape(np.array(data['comp']),(-1,1))
formation_energy = data['formation_energy']
name = data['name']

In [4]:
print(np.array(corr).shape)
print(np.array(eci_posterior).shape)


(67536, 253)
(253, 50000)


print(corr.shape)
comp = np.array(comp)
print(comp.shape)
formation_energy = np.array(formation_energy)
print(formation_energy.shape)
proposed_ground_state_indices = dj.clex.find_proposed_ground_states(corr, np.array(comp), np.array(formation_energy), eci_posterior.T)

with open('proposed_ground_state_indices.pkl', 'wb') as f:
    pickle.dump({'indices':proposed_ground_state_indices}, f, protocol=pickle.HIGHEST_PROTOCOL)


In [5]:
with open('proposed_ground_state_indices.pkl', 'rb') as f:
    proposed_ground_state_indices = pickle.load(f)
    proposed_ground_state_indices = proposed_ground_state_indices['indices']

In [6]:
eci_posterior.shape

(253, 50000)

In [7]:
#Quantify the new ground states that are appearing:
tally = np.bincount(proposed_ground_state_indices.astype(int))
unique_proposed_configs = np.unique(proposed_ground_state_indices.astype(int))

#Display statistics
config_percentage_dict = {}
config_percentage = []
proposed_names = []
for index, config in enumerate(unique_proposed_configs):
    print(name[config], end=' ')
    print('appears in %d of %d sample runs (%.6f percent)' % (tally[config], eci_posterior.shape[1], (100*tally[config]/eci_posterior.shape[1])) )
    config_percentage_dict["%s" % name[config]] = (100*tally[config]/eci_posterior.shape[1])
    config_percentage.append([config,(100*tally[config]/eci_posterior.shape[1])])
    proposed_names.append(name[config])

SCEL3_3_1_1_0_2_1/0 appears in 507 of 50000 sample runs (1.014000 percent)
SCEL4_2_2_1_0_1_0/0 appears in 65 of 50000 sample runs (0.130000 percent)
SCEL4_2_2_1_1_1_0/1 appears in 2 of 50000 sample runs (0.004000 percent)
SCEL4_4_1_1_0_0_2/0 appears in 40269 of 50000 sample runs (80.538000 percent)
SCEL5_5_1_1_0_1_3/0 appears in 4530 of 50000 sample runs (9.060000 percent)
SCEL6_3_2_1_0_1_2/1 appears in 3 of 50000 sample runs (0.006000 percent)
SCEL6_3_2_1_0_1_2/2 appears in 4027 of 50000 sample runs (8.054000 percent)
SCEL6_6_1_1_0_1_4/1 appears in 984 of 50000 sample runs (1.968000 percent)
SCEL6_6_1_1_0_3_2/7 appears in 6 of 50000 sample runs (0.012000 percent)
SCEL7_7_1_1_0_2_4/11 appears in 56 of 50000 sample runs (0.112000 percent)
SCEL7_7_1_1_0_3_5/0 appears in 12622 of 50000 sample runs (25.244000 percent)
SCEL7_7_1_1_0_3_5/1 appears in 421 of 50000 sample runs (0.842000 percent)
SCEL8_2_4_1_0_1_1/4 appears in 7173 of 50000 sample runs (14.346000 percent)
SCEL8_2_4_1_1_0_1/0 ap

In [8]:
config_percentage = np.array(config_percentage)

In [9]:
percentile = config_percentage[:,-1] > 75
config_percentage[percentile].shape
proposed_names = np.array(proposed_names)
proposed_names[percentile]

(11, 2)

array(['SCEL4_4_1_1_0_0_2/0', 'SCEL8_4_1_2_0_1_2/4',
       'SCEL8_8_1_1_0_5_2/10', 'SCEL10_5_1_2_0_2_3/7',
       'SCEL11_11_1_1_0_5_10/39', 'SCEL12_3_2_2_0_1_2/8',
       'SCEL12_12_1_1_0_9_3/9', 'SCEL14_7_1_2_0_6_3/73',
       'SCEL14_7_1_2_0_6_5/68', 'SCEL14_7_1_2_0_6_5/73',
       'SCEL15_15_1_1_0_0_13/391'], dtype='<U26')